In [3]:
import pygame
import sys
import os
import random
import time

pygame.init()
pygame.font.init()

# Constants
WIDTH, HEIGHT = 800, 800
INFO_HEIGHT = 100
MENU_HEIGHT = 400
SQUARE_SIZE = WIDTH // 8

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BROWN = (139, 69, 19)
LIGHT_BROWN = (181, 136, 99)
CREAM = (240, 217, 181)
YELLOW = (255, 255, 0)
GREEN = (0, 128, 0)
RED = (255, 0, 0)
GRAY = (200, 200, 200)
BLUE = (100, 149, 237)
DARK_BLUE = (70, 130, 180)

# Fonts
FONT = pygame.font.SysFont('Arial', 24)
LARGE_FONT = pygame.font.SysFont('Arial', 36)
TITLE_FONT = pygame.font.SysFont('Arial', 48, bold=True)

# Set up screen
screen = pygame.display.set_mode((WIDTH, HEIGHT + INFO_HEIGHT))
pygame.display.set_caption("♟️ Chess Game")

# Path to image directory
IMAGE_DIR = os.path.expanduser("~/OneDrive/Desktop/images")

class Button:
    def __init__(self, x, y, width, height, text, color, hover_color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover_color = hover_color
        self.is_hovered = False
        
    def draw(self):
        color = self.hover_color if self.is_hovered else self.color
        pygame.draw.rect(screen, color, self.rect, border_radius=10)
        pygame.draw.rect(screen, BLACK, self.rect, 2, border_radius=10)
        
        text_surface = LARGE_FONT.render(self.text, True, WHITE)
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)
        
    def check_hover(self, pos):
        self.is_hovered = self.rect.collidepoint(pos)
        
    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

# ChessPiece class
class ChessPiece:
    def __init__(self, color, type, image_name):
        self.color = color
        self.type = type
        image_path = os.path.join(IMAGE_DIR, image_name)
        try:
            self.image = pygame.image.load(image_path)
            self.image = pygame.transform.scale(self.image, (SQUARE_SIZE, SQUARE_SIZE))
        except pygame.error as e:
            # Create a fallback image with text
            self.image = pygame.Surface((SQUARE_SIZE, SQUARE_SIZE), pygame.SRCALPHA)
            text_color = WHITE if color == 'black' else BLACK
            text = FONT.render(type[0].upper(), True, text_color)
            text_rect = text.get_rect(center=(SQUARE_SIZE//2, SQUARE_SIZE//2))
            if color == 'black':
                pygame.draw.circle(self.image, BLACK, (SQUARE_SIZE//2, SQUARE_SIZE//2), SQUARE_SIZE//2 - 5)
            else:
                pygame.draw.circle(self.image, WHITE, (SQUARE_SIZE//2, SQUARE_SIZE//2), SQUARE_SIZE//2 - 5)
                pygame.draw.circle(self.image, BLACK, (SQUARE_SIZE//2, SQUARE_SIZE//2), SQUARE_SIZE//2 - 5, 2)
            self.image.blit(text, text_rect)
        self.has_moved = False
        # Keep track of piece value for AI evaluation
        piece_values = {'pawn': 1, 'knight': 3, 'bishop': 3, 'rook': 5, 'queen': 9, 'king': 100}
        self.value = piece_values[type]

# Game states
STATE_MENU = 0
STATE_PLAYING = 1
STATE_GAME_OVER = 2

# Initialize board
board = [[None for _ in range(8)] for _ in range(8)]

# Game state
game_state = STATE_MENU
current_player = 'white'
selected_piece = None
selected_pos = None
game_over = False
winner = None
check_status = False
show_instructions = False
valid_moves = []
game_mode = None  # 1 for single player, 2 for two player
ai_thinking = False
ai_color = 'black'  # AI plays as black in single player mode
last_move = None  # To highlight the last move
move_history = []  # To store moves for display and undo

# Initialize pieces on board
def init_board():
    for col in range(8):
        board[1][col] = ChessPiece('black', 'pawn', 'black_pawn.png')
        board[6][col] = ChessPiece('white', 'pawn', 'white_pawn.png')
    board[0][0] = board[0][7] = ChessPiece('black', 'rook', 'black_rook.png')
    board[7][0] = board[7][7] = ChessPiece('white', 'rook', 'white_rook.png')
    board[0][1] = board[0][6] = ChessPiece('black', 'knight', 'black_knight.png')
    board[7][1] = board[7][6] = ChessPiece('white', 'knight', 'white_knight.png')
    board[0][2] = board[0][5] = ChessPiece('black', 'bishop', 'black_bishop.png')
    board[7][2] = board[7][5] = ChessPiece('white', 'bishop', 'white_bishop.png')
    board[0][3] = ChessPiece('black', 'queen', 'black_queen.png')
    board[7][3] = ChessPiece('white', 'queen', 'white_queen.png')
    board[0][4] = ChessPiece('black', 'king', 'black_king.png')
    board[7][4] = ChessPiece('white', 'king', 'white_king.png')

# Draw menu
def draw_menu():
    screen.fill(DARK_BLUE)
    
    # Draw title
    title = TITLE_FONT.render("CHESS GAME", True, WHITE)
    screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 100))
    
    # Draw buttons
    single_player_btn = Button(WIDTH // 2 - 150, 250, 300, 80, "1 Player", BLUE, GREEN)
    two_player_btn = Button(WIDTH // 2 - 150, 350, 300, 80, "2 Player", BLUE, GREEN)
    instructions_btn = Button(WIDTH // 2 - 150, 450, 300, 80, "Instructions", BLUE, GREEN)
    exit_btn = Button(WIDTH // 2 - 150, 550, 300, 80, "Exit", RED, (200, 0, 0))
    
    mouse_pos = pygame.mouse.get_pos()
    
    for btn in [single_player_btn, two_player_btn, instructions_btn, exit_btn]:
        btn.check_hover(mouse_pos)
        btn.draw()
    
    return single_player_btn, two_player_btn, instructions_btn, exit_btn

# Draw chess board
def draw_board():
    for row in range(8):
        for col in range(8):
            color = WHITE if (row + col) % 2 == 0 else BROWN
            pygame.draw.rect(screen, color, (col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
    
    if selected_pos:
        pygame.draw.rect(screen, YELLOW, (selected_pos[1] * SQUARE_SIZE, selected_pos[0] * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))    
    # Highlight last move
    if last_move:
        from_row, from_col, to_row, to_col = last_move
        pygame.draw.rect(screen, BLUE, (from_col * SQUARE_SIZE, from_row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 3)
        pygame.draw.rect(screen, BLUE, (to_col * SQUARE_SIZE, to_row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 3)
    
    # Highlight selected piece
    if selected_pos:
        pygame.draw.rect(screen, YELLOW, (selected_pos[1] * SQUARE_SIZE, selected_pos[0] * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
        
        # Highlight valid moves
        for move in valid_moves:
            row, col = move
            # Use a different highlight if there's an opponent piece
            if board[row][col] and board[row][col].color != current_player:
                pygame.draw.rect(screen, RED, (col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 4)
            else:
                pygame.draw.rect(screen, GREEN, (col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 4)

# Draw pieces
def draw_pieces():
    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if piece:
                screen.blit(piece.image, (col * SQUARE_SIZE, row * SQUARE_SIZE))

# Draw info panel
def draw_info_panel():
    pygame.draw.rect(screen, GRAY, (0, HEIGHT, WIDTH, INFO_HEIGHT))
    
    if game_over:
        if winner:
            game_over_text = LARGE_FONT.render(f"CHECKMATE! {winner.upper()} WINS!", True, BLACK)
        else:
            game_over_text = LARGE_FONT.render("STALEMATE! GAME DRAWN", True, BLACK)
        
        restart_text = FONT.render("Press R to restart or M for menu", True, BLACK)
        
        screen.blit(game_over_text, (WIDTH // 2 - game_over_text.get_width() // 2, HEIGHT + 20))
        screen.blit(restart_text, (WIDTH // 2 - restart_text.get_width() // 2, HEIGHT + 60))
    else:
        # Show current player
        player_text = FONT.render(f"Current Turn: {current_player.capitalize()}", True, BLACK)
        screen.blit(player_text, (20, HEIGHT + 20))
        
        # Show game mode
        mode_text = FONT.render(f"Mode: {'AI opponent' if game_mode == 1 else 'Two player'}", True, BLACK)
        screen.blit(mode_text, (20, HEIGHT + 50))
        
        # Show AI thinking status
        if game_mode == 1 and ai_thinking and current_player == ai_color:
            ai_text = FONT.render("AI is thinking...", True, RED)
            screen.blit(ai_text, (WIDTH // 2 - ai_text.get_width() // 2, HEIGHT + 20))
        
        # Show check status
        if check_status:
            check_text = FONT.render(f"{current_player.capitalize()} is in CHECK!", True, RED)
            screen.blit(check_text, (WIDTH // 2 - check_text.get_width() // 2, HEIGHT + 50))
        
        # Show controls
        controls_text = FONT.render("H: Help | R: Restart | M: Menu | ESC: Quit", True, BLACK)
        screen.blit(controls_text, (WIDTH - controls_text.get_width() - 20, HEIGHT + 20))

# Draw instructions overlay
def draw_instructions():
    overlay = pygame.Surface((WIDTH, HEIGHT + INFO_HEIGHT), pygame.SRCALPHA)
    overlay.fill((0, 0, 0, 200))
    screen.blit(overlay, (0, 0))
    
    title = LARGE_FONT.render("Chess Game Instructions", True, WHITE)
    screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 40))
    
    instructions = [
        "- Click on a piece to select it",
        "- Green squares show valid moves",
        "- Red squares show pieces you can capture",
        "- Take turns moving pieces (white first, then black)",
        "- Capture your opponent's pieces by moving onto their square",
        "- Check occurs when a king is threatened",
        "- Checkmate is when a king cannot escape check",
        "- Pawns promote to queens when they reach the opposite end",
        "",
        "- In 1 Player mode, you play as white against the AI",
        "- In 2 Player mode, players take turns at the same computer",
        "",
        "- Press H to hide/show these instructions",
        "- Press R to restart the game",
        "- Press M to return to the main menu",
        "- Press ESC to quit the game"
    ]
    
    y_pos = 100
    for instruction in instructions:
        text = FONT.render(instruction, True, WHITE)
        screen.blit(text, (WIDTH // 2 - text.get_width() // 2, y_pos))
        y_pos += 30
    
    continue_text = FONT.render("Press any key to continue", True, WHITE)
    screen.blit(continue_text, (WIDTH // 2 - continue_text.get_width() // 2, HEIGHT + INFO_HEIGHT - 50))

# Get valid moves for a piece
def get_valid_moves(piece, row, col):
    moves = []
    if piece.type == 'pawn':
        direction = -1 if piece.color == 'white' else 1
        if 0 <= row + direction < 8 and board[row + direction][col] is None:
            moves.append((row + direction, col))
            if (piece.color == 'white' and row == 6) or (piece.color == 'black' and row == 1):
                if board[row + 2 * direction][col] is None:
                    moves.append((row + 2 * direction, col))
        for dc in [-1, 1]:
            if 0 <= row + direction < 8 and 0 <= col + dc < 8:
                if board[row + direction][col + dc] and board[row + direction][col + dc].color != piece.color:
                    moves.append((row + direction, col + dc))

    elif piece.type == 'rook':
        for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                if board[r][c] is None:
                    moves.append((r, c))
                elif board[r][c].color != piece.color:
                    moves.append((r, c))
                    break
                else:
                    break
                r += dr
                c += dc

    elif piece.type == 'knight':
        for dr, dc in [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2)]:
            r, c = row + dr, col + dc
            if 0 <= r < 8 and 0 <= c < 8 and (board[r][c] is None or board[r][c].color != piece.color):
                moves.append((r, c))

    elif piece.type == 'bishop':
        for dr, dc in [(1, 1), (1, -1), (-1, 1), (-1, -1)]:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                if board[r][c] is None:
                    moves.append((r, c))
                elif board[r][c].color != piece.color:
                    moves.append((r, c))
                    break
                else:
                    break
                r += dr
                c += dc

    elif piece.type == 'queen':
        for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                if board[r][c] is None:
                    moves.append((r, c))
                elif board[r][c].color != piece.color:
                    moves.append((r, c))
                    break
                else:
                    break
                r += dr
                c += dc

    elif piece.type == 'king':
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0:
                    continue
                r, c = row + dr, col + dc
                if 0 <= r < 8 and 0 <= c < 8 and (board[r][c] is None or board[r][c].color != piece.color):
                    moves.append((r, c))

    return moves

# Filter moves to prevent king from being in check
def filter_valid_moves(piece, row, col, moves):
    valid = []
    for move in moves:
        temp = board[move[0]][move[1]]
        board[move[0]][move[1]] = piece
        board[row][col] = None
        if not is_check(piece.color):
            valid.append(move)
        board[row][col] = piece
        board[move[0]][move[1]] = temp
    return valid

# Check if king is in check
def is_check(color):
    king_pos = None
    for r in range(8):
        for c in range(8):
            if board[r][c] and board[r][c].color == color and board[r][c].type == 'king':
                king_pos = (r, c)
                break
        if king_pos:
            break

    for r in range(8):
        for c in range(8):
            piece = board[r][c]
            if piece and piece.color != color:
                if king_pos in get_valid_moves(piece, r, c):
                    return True
    return False

# Check for checkmate or stalemate
def is_game_over():
    for r in range(8):
        for c in range(8):
            piece = board[r][c]
            if piece and piece.color == current_player:
                valid_moves = filter_valid_moves(piece, r, c, get_valid_moves(piece, r, c))
                if valid_moves:
                    return False
    return True

# Reset the game
def reset_game():
    global board, current_player, selected_piece, selected_pos, game_over, winner, check_status, valid_moves, last_move, move_history, ai_thinking
    board = [[None for _ in range(8)] for _ in range(8)]
    init_board()
    current_player = 'white'
    selected_piece = None
    selected_pos = None
    game_over = False
    winner = None
    check_status = False
    valid_moves = []
    last_move = None
    move_history = []
    ai_thinking = False

# Handle piece movement
def move_piece(from_row, from_col, to_row, to_col):
    global current_player, game_over, winner, check_status, last_move
    
    piece = board[from_row][from_col]
    captured_piece = board[to_row][to_col]
    
    # Check if we're capturing a king (shouldn't happen in real chess)
    if captured_piece and captured_piece.type == 'king':
        game_over = True
        winner = current_player
    
    # Move the piece
    board[to_row][to_col] = piece
    board[from_row][from_col] = None
    piece.has_moved = True
    
    # Record this move
    last_move = (from_row, from_col, to_row, to_col)
    move_history.append((from_row, from_col, to_row, to_col, captured_piece))

    # Promote pawn
    if piece.type == 'pawn' and (to_row == 0 or to_row == 7):
        board[to_row][to_col] = ChessPiece(piece.color, 'queen', f'{piece.color}_queen.png')

    # Switch player
    current_player = 'black' if current_player == 'white' else 'white'
    
    # Check for check
    check_status = is_check(current_player)
    
    # Check for checkmate or stalemate
    if is_game_over():
        game_over = True
        if check_status:
            winner = 'white' if current_player == 'black' else 'black'
        else:
            winner = None  # Stalemate

# Handle mouse click during gameplay
def handle_click(pos):
    global selected_piece, selected_pos, valid_moves
    
    # Ignore clicks if game is over, AI is thinking, or instructions are showing
    if game_over or (game_mode == 1 and current_player == ai_color) or show_instructions:
        return
        
    col = pos[0] // SQUARE_SIZE
    row = pos[1] // SQUARE_SIZE
    
    # Ignore clicks outside the chess board
    if not (0 <= row < 8 and 0 <= col < 8):
        return

    if selected_piece is None:
        piece = board[row][col]
        if piece and piece.color == current_player:
            selected_piece = piece
            selected_pos = (row, col)
            valid_moves = filter_valid_moves(piece, row, col, get_valid_moves(piece, row, col))
    else:
        if (row, col) == selected_pos:
            # Deselect if clicking the same piece
            selected_piece = None
            selected_pos = None
            valid_moves = []
        elif (row, col) in valid_moves:
            move_piece(selected_pos[0], selected_pos[1], row, col)
            selected_piece = None
            selected_pos = None
            valid_moves = []
        else:
            # If clicking another one of your pieces, select that piece instead
            piece = board[row][col]
            if piece and piece.color == current_player:
                selected_piece = piece
                selected_pos = (row, col)
                valid_moves = filter_valid_moves(piece, row, col, get_valid_moves(piece, row, col))
            else:
                # Clicked an invalid move, deselect
                selected_piece = None
                selected_pos = None
                valid_moves = []

# Evaluate board position for AI (simple version)
def evaluate_board():
    score = 0
    
    # Material value
    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if piece:
                value = piece.value
                if piece.color == ai_color:
                    score += value
                else:
                    score -= value
    
    # Bonus for controlling center
    center_squares = [(3, 3), (3, 4), (4, 3), (4, 4)]
    for row, col in center_squares:
        piece = board[row][col]
        if piece:
            bonus = 0.3  # Small bonus for center control
            if piece.color == ai_color:
                score += bonus
            else:
                score -= bonus
    
    # Bonus for checking opponent's king
    if is_check('white' if ai_color == 'black' else 'white'):
        score += 0.5
    
    # Penalty for being in check
    if is_check(ai_color):
        score -= 0.5
        
    return score

# Get all possible moves for a color
def get_all_moves(color):
    all_moves = []
    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if piece and piece.color == color:
                valid_moves = filter_valid_moves(piece, row, col, get_valid_moves(piece, row, col))
                for move in valid_moves:
                    all_moves.append((row, col, move[0], move[1]))
    return all_moves

# AI makes a move
def ai_move():
    global ai_thinking
    
    ai_thinking = True
    pygame.display.flip()  # Update display to show "AI thinking"
    
    # Add a small delay to make the AI seem like it's "thinking"
    time.sleep(0.5)
    
    all_moves = get_all_moves(ai_color)
    
    if not all_moves:
        ai_thinking = False
        return
    
    # Simple strategy: Evaluate each move and pick the best one
    best_score = -float('inf')
    best_moves = []
    
    for move in all_moves:
        from_row, from_col, to_row, to_col = move
        
        # Make the move temporarily
        piece = board[from_row][from_col]
        captured = board[to_row][to_col]
        board[to_row][to_col] = piece
        board[from_row][from_col] = None
        
        # Evaluate the resulting position
        score = evaluate_board()
        
        # Additional bonus for captures
        if captured:
            score += captured.value * 0.1
        
        # Undo the move
        board[from_row][from_col] = piece
        board[to_row][to_col] = captured
        
        # Keep track of the best move(s)
        if score > best_score:
            best_score = score
            best_moves = [move]
        elif score == best_score:
            best_moves.append(move)
    
    # Choose randomly among equally good moves
    chosen_move = random.choice(best_moves)
    from_row, from_col, to_row, to_col = chosen_move
    
    # Execute the move
    move_piece(from_row, from_col, to_row, to_col)
    
    ai_thinking = False

# Game loop
def main():
    global game_state, show_instructions, game_mode
    
    clock = pygame.time.Clock()
    running = True
    
    # Initialize the game board
    init_board()
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:  # Left click
                    if game_state == STATE_MENU:
                        single_player_btn, two_player_btn, instructions_btn, exit_btn = draw_menu()
                        pos = pygame.mouse.get_pos()
                        
                        if single_player_btn.is_clicked(pos):
                            game_mode = 1  # Single player
                            reset_game()
                            game_state = STATE_PLAYING
                        
                        elif two_player_btn.is_clicked(pos):
                            game_mode = 2  # Two player
                            reset_game()
                            game_state = STATE_PLAYING
                        
                        elif instructions_btn.is_clicked(pos):
                            show_instructions = True
                        
                        elif exit_btn.is_clicked(pos):
                            running = False
                    
                    elif game_state == STATE_PLAYING:
                        handle_click(pygame.mouse.get_pos())
            
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                
                elif event.key == pygame.K_r:
                    if game_state == STATE_PLAYING or game_state == STATE_GAME_OVER:
                        reset_game()
                        game_state = STATE_PLAYING
                
                elif event.key == pygame.K_m:
                    game_state = STATE_MENU
                
                elif event.key == pygame.K_h:
                    show_instructions = not show_instructions
                
                elif show_instructions:
                    show_instructions = False

        # Clear screen
        screen.fill(BLACK)
        
        # Draw appropriate screen based on game state
        if game_state == STATE_MENU:
            draw_menu()
        
        elif game_state == STATE_PLAYING or game_state == STATE_GAME_OVER:
            # Update game state if game is over
            if game_over:
                game_state = STATE_GAME_OVER
            
            # Draw game elements
            draw_board()
            draw_pieces()
            draw_info_panel()
            
            # AI move if it's AI's turn
            if game_mode == 1 and current_player == ai_color and not game_over and not ai_thinking:
                ai_move()
        
        # Draw instructions if needed
        if show_instructions:
            draw_instructions()
        
        pygame.display.flip()
        clock.tick(60)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

SystemExit: 

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
